In [ ]:
!pip install -U -q tqdm json_repair ddgs requests beautifulsoup4 SQLAlchemy wikipedia-api

In [ ]:
import requests
import zipfile
import io
import sys
import os
import shutil

# 1. The URL for the source code ZIP
url = "https://github.com/anpc849/kagentic/archive/refs/heads/main.zip"

# 2. Download the file using requests
print("Downloading...")
response = requests.get(url)
if response.status_code == 200:
    # 3. Unzip the content directly from memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(".")
    print("Unzipped successfully!")
else:
    print(f"Failed to download. Status code: {response.status_code}")

# 4. Add the folder to the Python Path
# GitHub's ZIPs always create a folder named 'repo-branch'
shutil.move(os.path.abspath('kagentic-main'), os.path.abspath('kagentic'))
module_path = "/kaggle/working"
if module_path not in sys.path:
    sys.path.append(module_path)

# 5. Verify the import
try:
    import kagentic
    print("Import successful! You can now use kagents.")
except ImportError:
    print("Import failed. Try checking !ls kagentic to see the folder structure.")

In [1]:
import kaggle_benchmarks as kbench
from kagentic import CodeAgent, Tool, ToolInput

In [2]:
MOCK_WEATHER_DB = {
    "hanoi":     {"condition": "Sunny", "temperature_c": 28},
    "tokyo":     {"condition": "Cloudy", "temperature_c": 14},
    "london":    {"condition": "Rainy",  "temperature_c": 10},
    "new york":  {"condition": "Windy",  "temperature_c": 5},
    "sydney":    {"condition": "Sunny",  "temperature_c": 31},
}


class WeatherTool(Tool):
    name = "get_weather"
    description = (
        "Returns the current weather for a given city. "
        "Use this to look up temperature and conditions."
    )
    inputs = {
        "city": ToolInput(
            type="string",
            description="Name of the city to get weather for (e.g. 'Tokyo', 'London').",
            required=True,
        )
    }
    output_type = "string"

    def forward(self, city: str) -> str:
        key = city.lower().strip()
        if key in MOCK_WEATHER_DB:
            data = MOCK_WEATHER_DB[key]
            return (
                f"Weather in {city.title()}: {data['condition']}, "
                f"{data['temperature_c']}°C"
            )
        return f"Sorry, no weather data available for '{city}'."

In [3]:
@kbench.task(
    name="tool_call_validation",
    description="Evaluate if the LLM Agents can generate a syntactically correct tool call."
)
def tool_call_validation(llm, question):
    agent = CodeAgent(
        tools=[WeatherTool()],
        model=llm,
        max_steps=25,
        verbosity_level=2,
    )

    print(f"Question: {question}\n")

    answer = agent.run(question)

    print("\n" + "="*60)
    print("FINAL ANSWER:")
    print(answer)
    print("="*60)

    kbench.assertions.assert_true(
        "14" in answer and "10" in  answer,
        expectation="Agent should return a non-empty answer about the weather.",
    )

In [4]:
tool_call_validation.run(llm=kbench.llm, question="What is the weather like in Tokyo and London?")

Question: What is the weather like in Tokyo and London?

[agent] 
[agent] 🤖 kagentic starting — model: google/gemini-2.5-flash
[agent] 📋 Task: What is the weather like in Tokyo and London?
[agent] ============================================================

[agent]   ⚠️  llm.prompt() attempt 1/3 failed: Response parsing failed.
Input Value:
---
```json
{
  "action": {
    "name": "get_weather",
    "arguments": {
      "city": "Tokyo"
    }
  }
}
```
---
Target Schema:
---
{
  "$defs": {
    "ToolCall": {
      "description": "A single tool invocation -- name + JSON-serialized argument string.",
      "properties": {
        "name": {
          "description": "Name of the tool to call. Use 'final_answer' when you have a complete answer for the user.",
          "title": "Name",
          "type": "string"
        },
        "arguments": {
          "default": "{}",
          "description": "Arguments to pass to the tool, as a JSON-encoded string. Example: {\"query\": \"hello world\"}. 

BokehModel(combine_events=True, render_bundle={'docs_json': {'e4d714b6-905f-4805-907b-81140e0c967b': {'version…